In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
df = pd.read_csv("modified_data/pol_svi_sc_merged.csv")
df.head()

,FIPS,3/31/21,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,...,Hopefulness,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct
0,1001,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,...,91.163142,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365
1,1003,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,...,82.484017,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,22.409030
2,1005,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,...,61.927181,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,45.788173
3,1007,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,...,85.258871,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,20.698280
4,1009,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,...,79.492703,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378


In [3]:
#set index to FIPS
df = df.set_index(df['FIPS'])
df= df.drop('FIPS', axis=1)

In [4]:
#rename target column
df = df.rename(columns={'3/31/21':'first_year_cases'})

In [5]:
#create variable for case% for each counties population
df['case_pct'] = df['first_year_cases']/df['E_TOTPOP']*100
df['case_pct'].head()

FIPS
1001    11.936594
1003     9.853104
1005     8.637809
1007    11.284237
1009    11.178767
Name: case_pct, dtype: float64

In [6]:
df['case_pct'].describe()

count    3058.000000
mean        9.426600
std         3.045809
min         0.000000
25%         7.713422
50%         9.466675
75%        11.176131
max        38.010657
Name: case_pct, dtype: float64

In [7]:
# bin and cut the case_pct column into 2 classifications
# q = df['case_pct'].quantile(.75)
q = df['case_pct'].quantile(.9)
bins = [0, q , 40]
labels = ['low','high']
df['case_class'] = pd.cut(df['case_pct'], bins, labels = labels)
df['case_class']

FIPS
1001      low
1003      low
1005      low
1007      low
1009      low
         ... 
56037     low
56039    high
56041     low
56043     low
56045     low
Name: case_class, Length: 3058, dtype: category
Categories (2, object): ['low' < 'high']

In [8]:
df['case_class'].value_counts()

low     2728
high     306
Name: case_class, dtype: int64

In [9]:
#drop unneeded columns
df = df.drop('case_pct', axis =1)
df

,first_year_cases,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,case_class
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,8050.0,...,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,low
1003,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,40665.0,...,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,22.409030,low
1005,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,4634.0,...,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,45.788173,low
1007,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,3661.0,...,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,20.698280,low
1009,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,10233.0,...,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,4721.0,...,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,22.894957,low
56039,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,3135.0,...,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,66.599040,high
56041,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,2498.0,...,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,16.819960,low


In [10]:
#turn case % classifications into binary 
df = pd.get_dummies(df, columns = ['case_class'])
df

,first_year_cases,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,...,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,8050.0,...,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,1,0
1003,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,40665.0,...,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,22.409030,1,0
1005,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,4634.0,...,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,45.788173,1,0
1007,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,3661.0,...,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,20.698280,1,0
1009,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,10233.0,...,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,4721.0,...,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,22.894957,1,0
56039,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,3135.0,...,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,66.599040,0,1
56041,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,2498.0,...,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,16.819960,1,0


## Start RF Regression

In [11]:
# Define feature set
X = df.copy()
X = df.drop(['case_class_low','case_class_high'], axis=1).values

In [12]:
# Define the target set.
y=df['case_class_high'].values
y[:3058]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [13]:
#split data for training and testing 
# making training set 70% of data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=78)
X_train.shape, X_test.shape

((2140, 103), (918, 103))

In [14]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2140, 103)
(918, 103)
(2140,)
(918,)


In [15]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)
predictions

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,751,67
Actual 1,67,33


### Sensitivty ~.9158, Specificity ~.6512, PPV ~ .9315, NPV ~.7000, Accuracy ~.8627

In [19]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [20]:
#(True Positives (TP) + True Negatives (TN)) 
Accuracy = (762 + 30)/918 
Accuracy

0.8627450980392157

In [21]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,751,67
Actual 1,67,33


Accuracy Score : 0.8540305010893247
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       818
           1       0.33      0.33      0.33       100

    accuracy                           0.85       918
   macro avg       0.62      0.62      0.62       918
weighted avg       0.85      0.85      0.85       918



In [22]:
# Calculate feature importance in the Random Forest model.
importances = model.feature_importances_
importances

array([0.08223093, 0.04139557, 0.04855143, 0.01039148, 0.0385237 ,
       0.        , 0.01315648, 0.00358095, 0.00923127, 0.00644572,
       0.        , 0.00358095, 0.        , 0.01639566, 0.01250244,
       0.        , 0.00268572, 0.01827682, 0.        , 0.        ,
       0.        , 0.00507302, 0.00447619, 0.00367279, 0.00447619,
       0.00819488, 0.00945174, 0.01120344, 0.00550916, 0.00127124,
       0.00930409, 0.00525207, 0.0367435 , 0.01007144, 0.03764786,
       0.00805715, 0.00831229, 0.        , 0.02106509, 0.01661617,
       0.00983773, 0.00925462, 0.01133233, 0.0035281 , 0.00472997,
       0.        , 0.01357246, 0.01259729, 0.        , 0.01262287,
       0.        , 0.00553006, 0.00739793, 0.        , 0.00429715,
       0.01705831, 0.02966998, 0.        , 0.01206396, 0.01572096,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [23]:
# We can sort the features by their importance.
X = df.drop(['case_class_low','case_class_high'], axis=1).columns
sorted(zip(model.feature_importances_, X), reverse=True)

[(0.08223092918481935, 'first_year_cases'),
 (0.04855142823755597, 'E_TOTPOP'),
 (0.04139556952937734, 'AREA_SQMI'),
 (0.041276903916183036, 'Employment Rate'),
 (0.03852370108307588, 'E_HH'),
 (0.03784904238353425, 'Agreeableness'),
 (0.03764786296742519, 'EP_GROUPQ'),
 (0.03674349584084523, 'EP_CROWD'),
 (0.029669979592069767, 'SPL_THEME4'),
 (0.021668468121820758, 'Tolerance'),
 (0.02106509254782008, 'EPL_NOHSDP'),
 (0.019981051611995305, 'Conscientiousness'),
 (0.018276820618162565, 'E_CROWD'),
 (0.017527421761898858, 'Belief In Science'),
 (0.017058308917754575, 'EPL_GROUPQ'),
 (0.016914887927180093, 'Conflict Awareness'),
 (0.01677895891078243, 'Openness'),
 (0.016616172264709864, 'SPL_THEME1'),
 (0.016395656765733364, 'E_MINRTY'),
 (0.01572095814638266, 'RPL_THEMES'),
 (0.015547882587054992, 'Neuroticism'),
 (0.01534565677997972, 'Collectivism'),
 (0.01529697212138217, 'Extraversion'),
 (0.013572457677562046, 'RPL_THEME2'),
 (0.013156478138536808, 'E_UNEMP'),
 (0.012622865232276